In [13]:
import sys, os
sys.path.append('/home/A00512318/TCN')
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from TCN.mnist_pixel.model import TCN
import numpy as np

In [15]:
from torchvision import datasets, transforms

def data_generator(root, batch_size):
    train_set = datasets.FashionMNIST(root=root, train=True, download=True,
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                               ]))
    test_set = datasets.FashionMNIST(root=root, train=False, download=True,
                              transform=transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.1307,), (0.3081,))
                              ]))

    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)
    return train_loader, test_loader

In [16]:
def trainTCN(ep):
    global steps
    train_loss = 0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda: 
            data, target = data.to(device), target.to(device)
        data = data.view(-1, input_channels, seq_length)
        if permutee:
            data = data[:, :, permute]
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        if clip > 0:
            torch.nn.utils.clip_grad_norm(model.parameters(), clip)
        optimizer.step()
        train_loss += loss.item()
        steps += seq_length
        if batch_idx > 0 and batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tSteps: {}'.format(
                ep, batch_idx * batch_size, len(train_loader.dataset),
                100. * batch_idx / len(train_loader), train_loss/log_interval, steps))
            train_losses_[ep-1].append(train_loss/log_interval)
            train_loss = 0

In [25]:
def testTCN():
    test_loss = 0
    correct = 0
    correct_class = np.zeros([10])
    correct_total = np.zeros([10])
    confusion_matrix = np.zeros([10, 10])
#     print(confusion_matrix)
    with torch.no_grad():
        for i, (data, target) in enumerate(test_loader):
            if cuda:
                data, target = data.to(device), target.to(device) # move devices to cuda
            data = data.view(-1, input_channels, seq_length)
            if permutee:
                data = data[:, :, permute]
#             data, target = Variable(data, volatile=True), Variable(target)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            _, pred = torch.max(output, 1)
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            c = (pred == target).squeeze()
#             c = (predicted == labels).squeeze()
#         for i in range(4):
#             label = labels[i]
#             class_correct[label] += c[i].item()
#             class_total[label] += 1
            size_check = c.size()[0]
#             print(size_check)
            for i in range(size_check):
#                 print(pred[i].item(), target.data.view_as(pred)[i].item())
#                 label = pred[i].item()
                label = target[i]
                correct_class[label] += c[i].item()
                correct_total[label] += 1
#                 print(label)
                t = pred[i].item()
                confusion_matrix[label][t] += 1
#                 print(i)
#                 if (label == t):
#                     print(c[i].item())
#                     correct_class[label] += c[i].item()
#                 correct_total[label] += 1

    test_loss /= len(test_loader.dataset)
    accuracies_.append(correct.item() / 10000.)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    print(confusion_matrix)
    for i in range(10):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * correct_class[i] / correct_total[i]))
    return test_loss, confusion_matrix

# testTCN()

In [18]:
import pickle

def save(model, file_name):
    torch.save(model, file_name)
    print('Saved as %s' % save_filename)
    
def save_obj(obj, name):
    with open('{0}.pkl'.format(name), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('{0}.pkl'.format(name), 'rb') as f:
        return pickle.load(f)
    
    
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)



model = TCN(input_channels, n_classes, hidden_units=nhid, levels=levels, 
            kernel_size=kernel_size, dropout=dropout)

count_parameters(model)

37010

In [31]:
# set default values for model
nummod = "1"
batch_size = 64
cuda = True
dropout = 0.0
clip = -1
epochs = 20
kernel_size = 6
levels = 8
log_interval = 100
lr = 2e-3
optimm = 'Adam'
nhid = 20
seed = 1111
torch.manual_seed(seed)
permutee = False
root = '../data/fashion_mnist'
input_channels = 1
n_classes = 10
seq_length = int(784 / input_channels)
steps = 0
file_name = './best_models/manual_{0}_20e.pt'.format(nummod)
classes = ('tshit/top', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle/boot')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = TCN(input_channels, n_classes, hidden_units=nhid, levels=levels, 
            kernel_size=kernel_size, dropout=dropout)
model = nn.DataParallel(model)
model.to(device)
optimizer = getattr(optim, 'Adam')(model.parameters(), lr=lr)
steps = 0

accuracies_ = []
train_losses_ = [[] for i in range(0, epochs)]
# test_losses_ = []
scores = []
train_loader, test_loader = data_generator(root, batch_size)

print("Model parameters size => {}".format(count_parameters(model)))

for epoch in range(1, epochs+1):
        trainTCN(epoch)
        testTCN()
        if epoch % 10 == 0:
#             print('Saving model.....')
#             torch.save({
#                 'epoch': epoch,
#                 'model_state_dict': model.module.state_dict(),
#                 'optimizer_state_dict': optimizer.state_dict(),
#                 'train_losses': train_losses_,
#                 'test_losses': test_losses_,
#                 'accuracies': accuracies_,
#                 'curr_lr': lr,
#             }, file_name)
            lr /= 10
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
torch.save(model.module.state_dict(), file_name)
scores.append({
    'accuracies': accuracies_,
    'train_losses': train_losses_,
#     'test_losses': test_losses_,
})
save_obj(scores, './best_models/manual_scores_{}_20e'.format(nummod))

Model parameters size => 37010
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.298547	Steps: 79184
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.701038	Steps: 157584
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.594310	Steps: 235984
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.527355	Steps: 314384
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.481741	Steps: 392784
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.458747	Steps: 471184
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.431339	Steps: 549584
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.432346	Steps: 627984
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.418199	Steps: 706384

Test set: Average loss: 0.4465, Accuracy: 8389/10000 (83%)

Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.396668	Steps: 814576
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.393481	Steps: 892976
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.395305	Steps: 971376
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.376072	Steps: 1049776
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.377098	Steps: 1128176
Train Epoch: 

Train Epoch: 13 [51200/60000 (85%)]	Loss: 0.197148	Steps: 9452688
Train Epoch: 13 [57600/60000 (96%)]	Loss: 0.186336	Steps: 9531088

Test set: Average loss: 0.2823, Accuracy: 9023/10000 (90%)

Train Epoch: 14 [6400/60000 (11%)]	Loss: 0.196576	Steps: 9639280
Train Epoch: 14 [12800/60000 (21%)]	Loss: 0.190447	Steps: 9717680
Train Epoch: 14 [19200/60000 (32%)]	Loss: 0.208149	Steps: 9796080
Train Epoch: 14 [25600/60000 (43%)]	Loss: 0.191463	Steps: 9874480
Train Epoch: 14 [32000/60000 (53%)]	Loss: 0.196568	Steps: 9952880
Train Epoch: 14 [38400/60000 (64%)]	Loss: 0.200821	Steps: 10031280
Train Epoch: 14 [44800/60000 (75%)]	Loss: 0.184032	Steps: 10109680
Train Epoch: 14 [51200/60000 (85%)]	Loss: 0.194020	Steps: 10188080
Train Epoch: 14 [57600/60000 (96%)]	Loss: 0.184548	Steps: 10266480

Test set: Average loss: 0.2820, Accuracy: 9037/10000 (90%)

Train Epoch: 15 [6400/60000 (11%)]	Loss: 0.192503	Steps: 10374672
Train Epoch: 15 [12800/60000 (21%)]	Loss: 0.187020	Steps: 10453072
Train Epoch: 15 

In [137]:
bst_model_filename = './best_models/paper_tunning_second_7.pt'
model = TCN(hidden_units=29, levels=10, kernel_size=6, dropout=0.0)
# {'batch_size': 64, 'dropout': 0.0, 'clip': -1.0, '
#  lr': 0.0009686968543691799, 'ksize': 6, 'levels': 10, 
#  'optim': 'Adam', 'nhid': 29, 'epochs': 20}
# TCN(input_channels, n_classes, hidden_units=nhid, levels=levels, 
#             kernel_size=kernel_size, dropout=dropout)
# best_model.torch.load(bst_model_filename, map_location="cuda:0")
model.load_state_dict(torch.load(bst_model_filename))  
model.to(device)
_, best_conf = testTCN()


Test set: Average loss: 0.2714, Accuracy: 9114/10000 (91%)

[[865.   0.  13.  26.   1.   2.  87.   0.   6.   0.]
 [  2. 986.   0.   6.   1.   0.   3.   0.   2.   0.]
 [ 15.   1. 880.  12.  49.   0.  42.   0.   1.   0.]
 [  4.   4.  13. 935.  21.   0.  21.   0.   2.   0.]
 [  1.   2.  54.  33. 869.   0.  40.   0.   1.   0.]
 [  0.   0.   0.   1.   0. 970.   0.  14.   0.  15.]
 [122.   2.  64.  35.  64.   0. 705.   0.   8.   0.]
 [  0.   0.   0.   0.   0.  15.   0. 948.   0.  37.]
 [  5.   1.   4.   1.   3.   1.   3.   2. 979.   1.]
 [  1.   1.   0.   0.   0.   5.   0.  16.   0. 977.]]
Accuracy of tshit/top : 86 %
Accuracy of trouser : 98 %
Accuracy of pullover : 88 %
Accuracy of dress : 93 %
Accuracy of  coat : 86 %
Accuracy of sandal : 97 %
Accuracy of shirt : 70 %
Accuracy of sneaker : 94 %
Accuracy of   bag : 97 %
Accuracy of ankle/boot : 97 %


In [27]:
# # batch_size = 64
# # cuda = True
# # dropout = 0.0
# # clip = -1
# # epochs = 20
# # kernel_size = 6
# # levels = 8
# # log_interval = 100
# # lr = 2e-3
# # optimm = 'Adam'
# # nhid = 20
# # seed = 1111
# # torch.manual_seed(seed)
# # permutee = False
# # root = '../data/fashion_mnist'
# # input_channels = 1
# # n_classes = 10
# # seq_length = int(784 / input_channels)
# # steps = 0
# my_best_model = './best_models/manual_1_20e.pt'
# model = TCN(hidden_units=20, levels=8, kernel_size=6, dropout=0.0)
# # {'batch_size': 64, 'dropout': 0.0, 'clip': -1.0, '
# #  lr': 0.0009686968543691799, 'ksize': 6, 'levels': 10, 
# #  'optim': 'Adam', 'nhid': 29, 'epochs': 20}
# # TCN(input_channels, n_classes, hidden_units=nhid, levels=levels, 
# #             kernel_size=kernel_size, dropout=dropout)
# # best_model.torch.load(bst_model_filename, map_location="cuda:0")
# model.load_state_dict(torch.load(my_best_model))  
# model.to(device)
# _, manual_conf = testTCN()



Test set: Average loss: 0.2844, Accuracy: 9045/10000 (90%)

[[858.   0.  15.  25.   2.   2.  95.   0.   3.   0.]
 [  0. 986.   0.   7.   1.   0.   6.   0.   0.   0.]
 [ 19.   3. 841.   9.  63.   0.  64.   0.   1.   0.]
 [ 12.   2.   9. 927.  28.   1.  20.   0.   1.   0.]
 [  1.   0.  62.  28. 853.   0.  55.   0.   1.   0.]
 [  0.   0.   0.   0.   0. 964.   0.  18.   0.  18.]
 [113.   0.  58.  29.  71.   1. 721.   0.   7.   0.]
 [  0.   0.   0.   0.   0.  15.   0. 952.   1.  32.]
 [  3.   2.   4.   6.   4.   2.   7.   2. 969.   1.]
 [  0.   0.   0.   0.   0.   4.   1.  21.   0. 974.]]
Accuracy of tshit/top : 85 %
Accuracy of trouser : 98 %
Accuracy of pullover : 84 %
Accuracy of dress : 92 %
Accuracy of  coat : 85 %
Accuracy of sandal : 96 %
Accuracy of shirt : 72 %
Accuracy of sneaker : 95 %
Accuracy of   bag : 96 %
Accuracy of ankle/boot : 97 %


In [138]:
manual_conf = np.array(manual_conf)
abs(best_conf - manual_conf)

array([[ 7.,  0.,  2.,  1.,  1.,  0.,  8.,  0.,  3.,  0.],
       [ 2.,  0.,  0.,  1.,  0.,  0.,  3.,  0.,  2.,  0.],
       [ 4.,  2., 39.,  3., 14.,  0., 22.,  0.,  0.,  0.],
       [ 8.,  2.,  4.,  8.,  7.,  1.,  1.,  0.,  1.,  0.],
       [ 0.,  2.,  8.,  5., 16.,  0., 15.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  6.,  0.,  4.,  0.,  3.],
       [ 9.,  2.,  6.,  6.,  7.,  1., 16.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,  1.,  5.],
       [ 2.,  1.,  0.,  5.,  1.,  1.,  4.,  0., 10.,  0.],
       [ 1.,  1.,  0.,  0.,  0.,  1.,  1.,  5.,  0.,  3.]])

In [269]:
# {'batch_size': 64, 'dropout': 0.0, 'clip': -1.0, 'lr': 0.0009686968543691799, 
# 'ksize': 6, 'levels': 10, 'optim': 'Adam', 'nhid': 29, 'epochs': 20}
len(model.tcn.network)
# for temporal_block in model.tcn.network:
#     print(temporal_block)
fst_layer = model.tcn.network[-1].conv2.weight.data.cpu()
fst_layer = fst_layer.numpy()
# num_kernels = fst_layer.shape[0]

# onv1 = nn.Conv2d(3, 1, 3)
# weight = conv1.weight.data.numpy()
# plt.imshow(fst_layer[0, ...])
# layers = model.tcn.network.children()
# for l in layers:
#     print(l.net)
    
layers = [l.net for l in model.tcn.network.children()]
for l in layers:
    print(l[0])

Conv1d(1, 29, kernel_size=(6,), stride=(1,), padding=(5,))
Conv1d(29, 29, kernel_size=(6,), stride=(1,), padding=(10,), dilation=(2,))
Conv1d(29, 29, kernel_size=(6,), stride=(1,), padding=(20,), dilation=(4,))
Conv1d(29, 29, kernel_size=(6,), stride=(1,), padding=(40,), dilation=(8,))
Conv1d(29, 29, kernel_size=(6,), stride=(1,), padding=(80,), dilation=(16,))
Conv1d(29, 29, kernel_size=(6,), stride=(1,), padding=(160,), dilation=(32,))
Conv1d(29, 29, kernel_size=(6,), stride=(1,), padding=(320,), dilation=(64,))
Conv1d(29, 29, kernel_size=(6,), stride=(1,), padding=(640,), dilation=(128,))
Conv1d(29, 29, kernel_size=(6,), stride=(1,), padding=(1280,), dilation=(256,))
Conv1d(29, 29, kernel_size=(6,), stride=(1,), padding=(2560,), dilation=(512,))


In [212]:
def plot_kernels(tensor, num_cols=6):
    num_kernels = tensor.shape[0]
#     print(num_kernels)
    print(tensor.shape)
    num_rows = 1+ num_kernels // num_cols
#     print(num_rows)
    fig = plt.figure(figsize=(num_cols,num_rows))
    len(tensor)
    for i in range(num_kernels):
#         print()
        ax1 = fig.add_subplot(num_rows,num_cols,i+1)
#         ax1.imshow(tensor[i][0,:], cmap='gray')
#         ax1.axis('off')
#         ax1.set_xticklabels([])
#         ax1.set_yticklabels([])
#     plt.subplots_adjust(wspace=0.1, hspace=0.1)
#     plt.show()

In [243]:
# filters = model.modules();
model.tcn.network.children()
model_layers = [i.net for i in model.tcn.network.children()];
model_layers
# plt.imshow(model_layers[0])
model_layers[0][0].weight.data.cpu()
model_layers
# len(model_layers)
# first_layer = model_layers[0];
# first_layer
# second_layer
# second_layer = model_layers[1];
# filtersl
# layers = [l for l in model_layers]
# layers
# kernels = [l[0].weight.data.cpu() for l in model_layers]
# kernels[0].numpy()[0][0]
# plot_kernels(kernels[0][0].numpy())
# kernels[0][0].numpy()[0][0,:]

[Sequential(
   (0): Conv1d(1, 29, kernel_size=(6,), stride=(1,), padding=(5,))
   (1): Chomp1d()
   (2): ReLU()
   (3): Dropout(p=0.0)
   (4): Conv1d(29, 29, kernel_size=(6,), stride=(1,), padding=(5,))
   (5): Chomp1d()
   (6): ReLU()
   (7): Dropout(p=0.0)
 ), Sequential(
   (0): Conv1d(29, 29, kernel_size=(6,), stride=(1,), padding=(10,), dilation=(2,))
   (1): Chomp1d()
   (2): ReLU()
   (3): Dropout(p=0.0)
   (4): Conv1d(29, 29, kernel_size=(6,), stride=(1,), padding=(10,), dilation=(2,))
   (5): Chomp1d()
   (6): ReLU()
   (7): Dropout(p=0.0)
 ), Sequential(
   (0): Conv1d(29, 29, kernel_size=(6,), stride=(1,), padding=(20,), dilation=(4,))
   (1): Chomp1d()
   (2): ReLU()
   (3): Dropout(p=0.0)
   (4): Conv1d(29, 29, kernel_size=(6,), stride=(1,), padding=(20,), dilation=(4,))
   (5): Chomp1d()
   (6): ReLU()
   (7): Dropout(p=0.0)
 ), Sequential(
   (0): Conv1d(29, 29, kernel_size=(6,), stride=(1,), padding=(40,), dilation=(8,))
   (1): Chomp1d()
   (2): ReLU()
   (3): Dropout

AttributeError: 'numpy.ndarray' object has no attribute 'sise'